In [ ]:
%pip install scikit-build packaging numpy scipy tqdm 

In [ ]:
!pip install --user Tasmanian --no-cache-dir

In [ ]:
from dataclasses import dataclass
from tqdm import tqdm
import numpy as np 
from scipy.stats import norm
import matplotlib.pyplot as plt
import Tasmanian

@dataclass()
class Option:
    S_0: float = 100
    K: float = 100
    r: float = 0.1
    T: float = 1
    sigma: float = 0.2
    epsilon: float = 1e-10
    d: int = 16

    def __post_init__(self):
        self.t_v = np.linspace(0, 1, self.d, endpoint=False) + 1 / self.d
        self.delta_t = self.t_v[0]
        # random walk brownian
        self.A = np.sqrt(self.delta_t) * np.tril(np.ones(self.d))

    def brownian_bridge_generator(self):
        st = np.mgrid[1 : self.d + 1, 1 : self.d + 1] / self.d
        cov = st.min(axis=0) * self.sigma**2
        return np.linalg.cholesky(cov)


@dataclass
class EuropeanOption(Option):
    S_0: float = 100
    K: float = 100
    r: float = 0.1
    sigma: float = 0.2
    epsilon: float = 1e-10
    d: int = 16

    def S_t(self, x: np.ndarray):
        """Calculate the asset price in the Black-Scholes model.

        Args:
            x (np.ndarray): Array with values in (0,1)

        Returns:
            np.ndarray: Asset price evolution.
        """
        S = self.S_0 * np.exp(
            (self.r - 0.5 * self.sigma**2) * self.T
            + self.sigma * np.sqrt(self.T)*(norm.ppf(x))
        )
        return S

    def payout_func(self):
        payout = np.exp(-self.r*self.T)*np.maximum(0, self.S_t(self.T) - self.K)
        return payout

    def scholes_call(self, St=None, t=0):
        if St is None and t==0:
            St=self.S_0
        tt_maturity = T-t
        d1 = 1/(self.sigma*np.sqrt(tt_maturity))*(
            np.log(St/self.K)+(self.r+0.5*self.sigma**2)*tt_maturity
        )
        d2 = d1-self.sigma*np.sqrt(tt_maturity)
        return norm.cdf(d1)*St-norm.cdf(d2)*self.K*np.exp(-self.r*tt_maturity)

@dataclass()
class AsianOption(Option):
    S_0: float = 100
    K: float = 100
    r: float = 0.1
    sigma: float = 0.2
    T: float = 1
    d: int = 16
    level: int = 3
    epsilon: float = 1e-10
    random_walk: bool = False

    def __post_init__(self):
        self.t_v = np.linspace(0, 1, self.d, endpoint=False) + 1 / self.d
        self.delta_t = self.t_v[0]

        # random walk brownian
        self.A = np.sqrt(self.delta_t) * np.tril(np.ones(self.d))

        self.M = np.log(self.S_0)+0.5*(self.r-.5*self.sigma**2)*(self.T+self.delta_t)
        self.denom = 1/((2*np.arange(self.d)+1)**2*np.pi**2)
        # gamma_d coefficients in PCA case
        self.gamma_d = 4*self.sigma*np.sqrt(2*self.T)*self.denom + \
        3*self.sigma*np.sqrt(2*self.T)*self.denom/self.d
        if self.random_walk:
            self.gamma_d = (-np.arange(1,9)+self.d+1)*self.sigma*np.sqrt(T)/(self.d*np.sqrt(self.d))
             
    
    def S_t(self, x: np.ndarray) -> np.ndarray:
        return self.S_0 * np.exp(
            (self.r - 0.5 * self.sigma**2) * self.t_v
            + self.sigma * self.A@(norm.ppf(x))
        )

    def brownian_bridge_generator(self) -> np.ndarray:
        st = np.mgrid[1:self.d+1, 1:self.d+1]/self.d
        cov = st.min(axis=0)*self.sigma**2
        A = np.linalg.cholesky(cov) # Brownian Bridge brownian
        return A

    def payout_func(self, x: np.ndarray) -> float:
        """Geometric Asian payoff
        """
        payout = np.maximum(
            0, (np.prod([x[i] for i in range(len(x))])** (1 / self.d)) - self.K
        )
        return payout

    def payout_func_opt(self, x: np.ndarray)-> float:
        """Payout array for array x with values in [0,1]^d without pre-factor
        """
        payout_v = np.maximum(
        np.exp(norm.ppf(x)@self.gamma_d) - np.exp(-self.M)*self.K, 0
    )
        return payout_v

    def payout_func_opt_der(self, x: np.ndarray, coordinate=0):
        """Coordinate-wise derivative of payout_func_opt.
        """
        payout_der = np.exp(-self.r*self.T+self.M) * np.maximum(0, np.exp(
            np.inner(self.gamma_d, norm.ppf(x))
        )* self.gamma_d[coordinate]/norm.pdf(norm.ppf(x[coordinate])))

        return payout_der

    def gen_quad(self, lb=0, ub=1, rule=None):
        """Quadrature node and weight generator.

        Args:
            lb (int, optional): [description]. Defaults to 0.
            ub (int, optional): [description]. Defaults to 1.
            rule (str, optional): Name of the quadrature rule.
                For rules refer to the chaospy.quadrature module.
                Defaults to None.

        Returns:
        (numpy.ndarray, numpy.ndarray):
            Abscissas and weights created from full tensor grid rule. Flatten
            such that ``abscissas.shape == (len(dist), len(weights))``.
        """
        iid_dist = cp.Iid(cp.Uniform(lb + self.epsilon, ub - self.epsilon), self.d)
        return cp.generate_quadrature(self.level, iid_dist, sparse=True, rule=rule)

    def compute_premium(self):
        abscissas, weights = self.gen_quad()
        payout_v = self.payout_func(abscissas)
        return np.dot(weights, payout_v) * np.exp(-self.r * self.T)
    
    def scholes_call(self, t=0) -> float:
        t1 = self.T - (
            self.d*(self.d-1)*(4*self.d+1)*self.delta_t
        ) / (6*self.d**2)
        t2 = self.T-0.5*(self.d-1)*self.delta_t
        beta = (
            np.log(self.S_0/self.K)+(self.r-0.5*self.sigma**2)*t2
        ) / (self.sigma*np.sqrt(t1))
        gamma = np.exp(
            -self.r*(self.T-t2)-0.5*self.sigma**2*(t2-t1)
        )
        return (
            self.S_0*gamma*norm.cdf(beta+self.sigma*np.sqrt(t1)) -
            self.K*np.exp(-self.r*self.T)*norm.cdf(beta)
        )
        

def sde_body(idx, s, sigma,mu,T,K, samples, batch_size, N=128): 
    h=T/N
    z=np.random_normal(shape=(samples, batch_size,1),
                          stddev=1., dtype=np.float32)
    s=s + mu *s * h +sigma * s *np.sqrt(h)*z + 0.5 *sigma *s *sigma * ((np.sqrt(h)*z)**2-h)    
    return s

def make_grid(dim, level, lb, rb, rule):
    """Creates a sparse grid according to given rules and boundaries.

    Args:
        dim (int): Input Dimension.
        level (int): level of the integration.
        lb (array-like): Left boundary. Shape (dim) is recommended.
        rb (array-like): Right boundary. Shape (dim) is recommended.
        rule (str):  One of the local polynomial rules in TASMANIAN docs.
            Defaults to "localp".

    Returns:
        TasmanianSparseGrid: SparseGrid object.
    """
    grid = Tasmanian.makeGlobalGrid(
        dim, 1, level, "level", rule
    )
    grid.setDomainTransform(np.vstack([lb, rb]).T)
    return grid

s_0_l=80.0
s_0_r=120.0
sigma_l=0.1
sigma_r=0.2
mu_l=0.02
mu_r=0.05
T_l=0.9
T_r=1.0
K_l=109.0
K_r=110.0
# %%
# max possible dim, level before 16 GB RAM is full: 40, 4
dim = 16
aop = AsianOption(d=dim)
level = 4
lb, rb = np.zeros(dim), np.ones(dim)
grid = make_grid(dim, level, lb, rb, "gauss-patterson")
weights, points = grid.getQuadratureWeights(), grid.getPoints()
pre_factor = np.exp(-aop.r*aop.T+aop.M)
payout_v = aop.payout_func_opt(points)
pre_factor*np.inner(weights, payout_v), aop.scholes_call()
# %%
grid2d = make_grid(2, 2, np.zeros(2), np.ones(2), "gauss-patterson")
weights2d, points2d = grid2d.getQuadratureWeights(), grid2d.getPoints()
plt.scatter(points2d[:, 0], points2d[:, 1])
plt.title("Gauss-Patterson Sparse Grid")
#plt.savefig("Gauss_patterson_sg.PNG")
# %%
s_0 = np.linspace(s_0_l, s_0_r, 21)
sigma = np.linspace(sigma_l, sigma_r, 5)
mu = np.linspace(mu_l, mu_r, 9)
T = np.linspace(T_l, T_r, 9)
K = np.linspace(K_l, K_r, 9)
# %%
mesh = np.meshgrid(s_0, sigma, mu, T, K)
grid_ar = np.stack(mesh, -1).reshape(-1, 5)
## vectorizing training data generation takes up too much memory
# -> make a loop
#for s_0, sigma, mu, T, K in grid_ar[:10]:
#    print(s_0, sigma, mu, T, K)
# %%
#grid_ar.shape[0]
Y = np.empty((grid_ar.shape[0], 2), dtype=np.float64)
i = 0
for s_0, sigma, mu, T, K in tqdm(grid_ar):
    aop = AsianOption(S_0=s_0, sigma=sigma, r=mu, T=T, K=K, d=dim)
    pre_factor = np.exp(-aop.r*aop.T+aop.M)
    payout_v = aop.payout_func_opt(points)
    Y[i] = pre_factor*np.inner(weights, payout_v), aop.scholes_call()
    i=+1